In [3]:
import pickle
import numpy as np
import string
import operator
import nltk
import gensim

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as auc

In [2]:
data = pickle.load(open('../saved_files/clinical_notes_excluding_newborns.p', 'rb'))
label = pickle.load(open('../saved_files/mortality_excluding_newborns.p', 'rb'))
print len(data), len(label)

51113 51113


In [156]:
admission_id = pickle.load(open('../saved_files/admission_id_los.p', 'rb'))
remove_id = [100795, 145875, 128483, 138159, 119477, 133974, 124330, 161603, 185708, 145084, 138279, 107872, 
             154213, 137453, 171190, 170587, 179637, 175992, 120139, 129842, 163762, 182711, 151775, 111089,
             197028, 121639, 178442, 153970, 151840, 147340, 127108, 191510, 104737, 189207, 100373, 139697,
             134534, 168470, 111233, 143882, 100085, 111736, 143290, 177563, 117358, 150972, 139969, 148815, 
             147134, 136735, 196038, 189920, 144395, 131896, 190460, 135303, 180686, 133651, 137889, 161709, 
             130833, 191607, 104268, 171324, 120933, 128104, 154857, 132089, 142897, 116501, 173944, 176063, 
             140097, 160510, 123712, 130183, 196984, 129791, 156454, 134966, 112235, 145595, 102591, 186211, 
             110600, 137516, 165586, 181396, 106378, 169529, 192679, 169423, 141885, 156845, 169685]


remove_indices = []
final_id_set = [] 
for i, record in enumerate(label):
    if record[0] in admission_id and record[0] not in remove_id and len(admission_data2[i]):
        final_id_set.append(record[0])
    else:
        remove_indices.append(i)
print(len(final_id_set))

45202


In [4]:
def custom_split(s):
    indices = []
    unique_inds = []
    splits = []
    keep_track = 0
    for i in range(len(s)):
        if s[i] == '\n':
            keep_track = i
        elif s[i] == ':':
            indices.append(keep_track)
    if len(indices) > 0:
        unique_inds.append(indices[0])
        for i in range(1,len(indices)):
            if indices[i] != indices[i-1]:
                unique_inds.append(indices[i])

        for i in range(len(unique_inds)-1):
            splits.append(s[unique_inds[i]: unique_inds[i+1]])
        splits.append(s[unique_inds[-1]:])
    return splits


In [ ]:
# admission_data = []
# admission_info = ['history of present illness:', 'hpi:', 'chief complaint:', 'past medical history:', 
#                   'major surgical or invasive procedure:']
# i = 0
# for record in data:
#     combined_info = []
#     headings = record.split('\n\n')
#     for head in headings:
#         head = head.lower()
#         for info in admission_info:
#             if info in head:
#                 token = head.split(':')
#                 combined_info.append(token[1])
#     print i
#     i+= 1
#     admission_data.append(combined_info)
# print len(admission_data)

In [178]:
def remove_special_chars(s):
    init = -1
    s = list(s)
    for i, ch in enumerate(s):
        if ch == '[':
            init = i
        elif ch == ']' and init > 0:
            s[init:i] = ''
            init = -1
    s = "".join(s)
    return s

In [167]:
count = [0]*len(admission_info)
for record in data:
    for i,info in enumerate(admission_info):
        if info in record.lower():
            count[i] +=1
print count


[47478, 1367, 40707, 47238, 38140, 46340, 43756, 40263, 225]


In [168]:
count = 0
for record in data:
    if len(record) == 0:
        count+=1.0
print '% records with no clinical notes:',100*count/len(data), count
#print '% positive labels:', 100.0*np.sum(y)/y.shape[0]

count = 0
for i, record in enumerate(admission_data2):
    if len(record) == 0 and len(data[i]) != 0:
        count+=1
print '% records with no admission data:', 100.0*count/len(data)

% records with no clinical notes: 3.9715923542 2030.0
% records with no admission data: 0.739537886643


In [179]:
admission_data3 = []
admission_info = ['history of present illness:', 'hpi:', 'chief complaint:', 'past medical history:', 
                  'major surgical or invasive procedure:', 'allergies:', 'social history:',  
                  'family history:', 'family hx:']
i = 0
for record in data:
    combined_info = []
    headings = custom_split(record)
    for head in headings:
        head = head.lower()
        for info in admission_info:
            if info in head:
                token = head.split(':')
                for sent in nltk.sent_tokenize(remove_special_chars(token[1])):
                    combined_info.append(sent)
    admission_data3.append(combined_info)

In [187]:
temp = []
input_dataset = []
for record in admission_data3:
    record_embedding = []
    if len(record) > 0:
        for sent in record:
            sent_embedding =[]
            punc_list = string.punctuation+'\n'+'0123456789' 
            t = string.maketrans(punc_list, " "*len(punc_list))
            sent = sent.translate(t)
            temp.append(sent)
            for word in word_tokenize(sent):
                if word in embeddings:
                    sent_embedding.append(embeddings[word])
            if len(sent_embedding) > 0: 
                record_embedding.append(np.mean(sent_embedding, axis=0))
    input_dataset.append(record_embedding)

In [206]:
input_dataset[0]

[array([-2.3183e-01,  5.0164e-01,  7.7314e-02,  4.2377e-01, -2.9015e-01,
        -2.2334e-01,  2.8047e-01,  7.9432e-03, -4.8036e-02, -5.4698e-01,
         7.6200e-01,  1.1681e-01,  2.8522e-01, -1.7298e-01,  3.4805e-01,
        -2.3633e-01,  7.2703e-02, -7.6276e-01,  1.8180e-01,  2.0065e-01,
        -3.4172e-01, -3.1133e-04, -5.3486e-01, -5.1988e-01,  5.2648e-01,
         6.3894e-02, -1.0303e-01,  3.6646e-01,  1.7052e-01,  3.3825e-01,
         1.6559e-01, -1.0657e-01, -7.9700e-01, -4.6131e-02, -2.4400e-01,
         1.4344e-01, -4.4849e-01,  3.5055e-01, -3.9082e-02,  2.0014e-01,
         5.4479e-01, -8.2090e-01, -6.9253e-01, -7.1692e-02, -2.2485e-01,
         2.2833e-01,  1.5706e-01,  1.7680e-01,  2.1599e-01, -7.3419e-02,
         8.0549e-01, -7.5346e-02,  8.9822e-03,  4.7648e-02, -4.1519e-02,
         2.0390e-01, -7.3100e-02, -6.7930e-01, -9.5766e-02,  1.9383e-01,
         1.5077e-01,  8.0904e-01, -2.2470e-01, -5.7263e-02, -5.8042e-01,
        -4.4154e-01,  2.1145e-01,  4.3674e-03, -5.1

In [208]:
len_data = []
for i in range(len(input_dataset)):
    len_data.append(len(input_dataset[i]))
np.histogram(len_data)

(array([14812, 23790,  9929,  2082,   360,    88,    32,    11,     7,
            2]),
 array([  0.,  12.,  24.,  36.,  48.,  60.,  72.,  84.,  96., 108., 120.]))

(51113,)

In [93]:
lemmatizer = WordNetLemmatizer()

def preprocess(s):
    #removing the names/dates in []
#     init = -1
#     s = list(s)
#     for i, ch in enumerate(s):
#         if ch == '[':
#             init = i
#         elif ch == ']' and init > 0:
#             s[init:i] = ''
#             init = -1
#     s = "".join(s)
    
    if len(s)>0:
        punc_list = string.punctuation+'\n'+'0123456789'
        t = string.maketrans(punc_list, " "*len(punc_list))
        s = s.translate(t)
    tokens = ''
    for word in word_tokenize(s):
        if word not in 'abcdefghijklmnopqrstuvwxyz':
            #word = lemmatizer.lemmatize(word)
            tokens += word + ' '
    return tokens

In [94]:
preprocessed_data = []
for record in admission_data2:
    preprocessed_data.append(preprocess(record))

In [12]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('../saved_files/GoogleNews-vectors-negative300.bin', binary=True) 

In [19]:
l = []
count1, count2= 0,0
for record in preprocessed_data:
    for word in word_tokenize(record):
        flag1 = word not in model.vocab
        flag2 = word not in l
        if flag1 and flag2:
            count2 +=1
            l.append(word)
            count1 +=1
        elif flag2:
            l.append(word)
            count1 +=1
        
print count1, count2


64173 40352


In [96]:
l = {}
for record in preprocessed_data:
    for word in word_tokenize(record):
        if word not in l:
            l[word] = 1.0
        else:
            l[word] += 1.0
len(l)

55482

In [97]:
count =0
oov_words = {}
for word in l:
    if word not in embeddings:
        oov_words[word] = l[word]
        count +=1
print count       

22168


In [99]:
occurences = []
#sorted_oov = sorted(oov_words.items(), key=operator.itemgetter(1))
for key in oov_words:
    occurences.append(oov_words[key])
np.histogram(occurences, bins=10, range=(0, 1000))

(array([22053,    56,    23,     7,     4,     6,     1,     5,     1,
            1]),
 array([   0.,  100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,
         900., 1000.]))

In [41]:
embeddings ={}
f = open('../saved_files/glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

In [52]:
model2 = gensim.models.KeyedVectors.load_word2vec_format('../saved_files/bio_nlp_vec/PubMed-shuffle-win-2.bin', binary=True)
embeddings2 ={}
f = open('../saved_files/stanford_cuis_svd_300.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings2[word] = coefs
f.close()

In [107]:
#input_data = np.zeros((len(preprocessed_data), 300))
input_data=[]
for i, record in enumerate(preprocessed_data):
    avg = [0]*300
    count = 0.0
    if len(record) == 0:
        input_data.append(np.array(avg))
    else:
        tokens = word_tokenize(record)
        for word in tokens:
            if word in embeddings:
                avg += embeddings[word]
                count +=1.0
        input_data.append(np.array(avg)/count)

input_data = np.array(input_data)
input_data.shape
        

(51113, 300)

In [160]:
y = np.array(label)
y = y[:,1]

x = np.delete(input_data, remove_indices, axis = 0) 
y = np.delete(y, remove_indices, axis = 0) 
print x.shape, y.shape
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=0)

(45202, 300) (45202,)


In [136]:
clf=LogisticRegression(C = 5.0, penalty = 'l2', random_state = 0)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_val)
y_pred = y_pred[:,1]
print auc(y_val, y_pred)

0.760803739900028


In [137]:
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]
print auc(y_test, y_pred)

0.7616190685768776


In [143]:
max_len = 0
length = []
for record in preprocessed_data:
    record_len = len(word_tokenize(record))
    length.append(record_len)
    if max_len < record_len:
        max_len = record_len
print max_len
    

1664


In [144]:
np.histogram(length)

(array([23646, 20128,  6074,   970,   203,    49,    22,    10,     6,
            5]),
 array([   0. ,  166.4,  332.8,  499.2,  665.6,  832. ,  998.4, 1164.8,
        1331.2, 1497.6, 1664. ]))

In [149]:
count = 0
for t in length:
    if t>700:
        count+=1
count

230

In [158]:
count=0
for record in admission_data2:
    if len(record) == 0:
        count+=1
print count

2408
